In [7]:
%run generate_new_examples.ipynb     # importing functions

[ 11   8  48]
[-16 -44  35]
[  0  14   5] :  B
[2489 1152  352]
[1152 3417 -441]
[ 352 -441  221] :  G
(0, 0, 1) :  lincomb_LLL
(-0.23835, 0.20942, 1.0000) :  lincomb_cube
(0, 14, 5) :  sv_LLL
(-5.9725, 2.8789, 0.88888) :  sv_cube
(0.23835, -0.20942, 0.00000) :  lincomb_diff
[ 141.4  -57.5    0.0]
[ -57.5 149.85    0.0]
[   0.0    0.0    0.0] :  Diff impact

[ 14  48  19]
[ 27  30 -34]
[ 15  22 -10] :  B
[2861 1172 1076]
[1172 2785 1405]
[1076 1405  809] :  G
(1, 2, -5) :  lincomb_LLL
(1.0000, 2.0199, -4.8380) :  lincomb_cube
(-7, -2, 1) :  sv_LLL
(-4.0330, 2.1606, -1.2964) :  sv_cube
(0.00000, -0.019897, -0.16199) :  lincomb_diff
[  0.0  -0.0  -0.0]
[ -0.0   1.1  4.53]
[ -0.0  4.53 21.23] :  Diff impact

[ 31 -35 -17]
[-19  17 -36]
[ 21  27 -38] :  B
[2475 -572  352]
[-572 1946 1428]
[ 352 1428 2614] :  G
(0, -1, 1) :  lincomb_LLL
(-0.31484, -1.0000, 0.58869) :  lincomb_cube
(40, 10, -2) :  sv_LLL
(21.602, 9.9137, 18.982) :  sv_cube
(0.31484, 0.00000, 0.41131) :  lincomb_diff
[245.32 

In [8]:
# jsonfilename = "noname.json"
limits = [0.05, 0.005, 0.0010, 0.0005, 0.0001]

# Rounding detection

This notebook's purpose is to test a following hypothesis: given a "cube" defined by real-valued coordinates from the cube algorithm, assume a rounding of these coordinates, which are in some sense *nearly* an integer. We test whether this rounding gives us the desired coordinate of the linear combination given by the LLL. 

Note: this may not work always, as sometimes the jump is greater than 1.

In [9]:
def detect_rounding(jsonfilename):
    '''
    OUTPUT: dict in the form \{ limit: [succesful attempts, unsuc att.],...\}
    '''
    cases = from_json(jsonfilename)
    stats = {}
    for limit in limits:
        stats[limit] = [0, 0, 0, 0] 
    for case in cases:
        for i in range(len(case["lincomb_cube"])):
            
            #skip equal components
            if case["lincomb_diff"][i] == 0:
                continue
                
            for limit in limits:
                num_cube = case["lincomb_cube"][i]
                num_LLL = case["lincomb_LLL"][i]
                is_near_int, direction =  is_nearest_integer(num_cube, limit)
                if is_near_int: # if its almost an int
                    if round(num_cube) == num_LLL: 
                        stats[limit] = increment(stats, limit, 0)                        # rounding lead directly to lcLLL
                    else:
                        stats[limit] = increment(stats, limit, 1)                        # rounding didnt lead directly to lcLLL
                        if lcLLL_is_in_same_direction(direction, num_cube, num_LLL):
                            stats[limit] = increment(stats, limit, 2)                    # but it was in the same direction
                        else: stats[limit] = increment(stats, limit, 3)
    return stats

def is_nearest_integer(num, tolerance):
    nearest_int = round(num)
    is_within_tolerance = abs(num - nearest_int) <= tolerance
    if not is_within_tolerance:
        return False, None
    direction = "up" if num - nearest_int < 0 else "down"
    return True, direction


def increment(dic, limit, ind):
    lst = [dic.get(limit,0)[0], dic.get(limit,0)[1],  dic.get(limit,0)[2],  dic.get(limit,0)[3]]
    lst[ind] += 1
    return lst

def lcLLL_is_in_same_direction(direction, num_cube, num_LLL, ):
    return abs(num_cube - num_LLL) - abs(round(num_cube) - num_LLL) > 0


In [85]:
for [limit, [a,b,c,d]] in detect_rounding("matrices6x6-12000instances.json").items():
    print("success: {}  fail: {} for limit = {}".format(a,b, limit.n(digits=1)))
    print("successful rounding ratio: ", 100 *a/(a + b).n(digits=3), " % out of all nearly integers")
    print()
    
for [limit, [a,b,c,d]] in detect_rounding("matrices6x6-12000instances.json").items():
    print(limit, a, b,c,d)

success: 338  fail: 2430 for limit = 0.050
successful rounding ratio:  12.2  % out of all nearly integers

success: 26  fail: 220 for limit = 0.0050
successful rounding ratio:  10.6  % out of all nearly integers

success: 8  fail: 44 for limit = 0.0010
successful rounding ratio:  15.4  % out of all nearly integers

success: 1  fail: 20 for limit = 0.00050
successful rounding ratio:  4.76  % out of all nearly integers

success: 0  fail: 6 for limit = 0.00010
successful rounding ratio:  0.000  % out of all nearly integers

0.0500000000000000 338 2430 1192 1238
0.00500000000000000 26 220 118 102
0.00100000000000000 8 44 21 23
0.000500000000000000 1 20 9 11
0.000100000000000000 0 6 2 4


In most cases, the success rate is about 10-15 \%, which is quite surprising. I thought that as the limit gets lower, fewer cases would success and the success rate would decrease. 

## Not wall, at least direction?


So now, we've 

## Does rounding work on functioning matrices?





In [10]:
functional_filename = "functioningMatrices3x3.json"


for [limit, [a,b,c,d]] in detect_rounding(functional_filename).items():
    print("success: {}  fail: {} for limit = {}".format(a,b, limit.n(digits=1)))
    print("successful rounding ratio: ", 100 *a/(a + b).n(digits=3), " % out of all nearly integers")
    print()
    
for [limit, [a,b,c,d]] in detect_rounding(functional_filename).items():
    print(limit, a, b,c,d)

success: 1258  fail: 16 for limit = 0.050
successful rounding ratio:  98.7  % out of all nearly integers

success: 127  fail: 1 for limit = 0.0050
successful rounding ratio:  99.2  % out of all nearly integers

success: 23  fail: 0 for limit = 0.0010
successful rounding ratio:  100.  % out of all nearly integers

success: 13  fail: 0 for limit = 0.00050
successful rounding ratio:  100.  % out of all nearly integers

success: 4  fail: 0 for limit = 0.00010
successful rounding ratio:  100.  % out of all nearly integers

0.0500000000000000 1258 16 6 10
0.00500000000000000 127 1 0 1
0.00100000000000000 23 0 0 0
0.000500000000000000 13 0 0 0
0.000100000000000000 4 0 0 0


In [11]:
functional_filename = "functioningMatrices6x6.json"


for [limit, [a,b,c,d]] in detect_rounding(functional_filename).items():
    print("success: {}  fail: {} for limit = {}".format(a,b, limit.n(digits=1)))
    print("successful rounding ratio: ", 100 *a/(a + b).n(digits=3), " % out of all nearly integers")
    print()
    
for [limit, [a,b,c,d]] in detect_rounding(functional_filename).items():
    print(limit, a, b,c,d)

success: 2224  fail: 52 for limit = 0.050
successful rounding ratio:  97.7  % out of all nearly integers

success: 217  fail: 5 for limit = 0.0050
successful rounding ratio:  97.8  % out of all nearly integers

success: 43  fail: 0 for limit = 0.0010
successful rounding ratio:  100.  % out of all nearly integers

success: 26  fail: 0 for limit = 0.00050
successful rounding ratio:  100.  % out of all nearly integers

success: 3  fail: 0 for limit = 0.00010
successful rounding ratio:  100.  % out of all nearly integers

0.0500000000000000 2224 52 5 47
0.00500000000000000 217 5 1 4
0.00100000000000000 43 0 0 0
0.000500000000000000 26 0 0 0
0.000100000000000000 3 0 0 0
